In [4]:
import os
from pathlib import Path
from aocd.models import Puzzle
import collections
import numpy as np
import re
from statistics import median, mean
import math
import queue
import itertools
import more_itertools
infinite_defaultdict = lambda: defaultdict(infinite_defaultdict)
from copy import copy
import functools
import heapq
import operator
import tqdm
import networkx as nx

# Day 1

In [4]:
puzzle = Puzzle(2024, 1)

In [7]:
data = [list(map(int, line.split("   "))) for line in puzzle.input_data.split("\n")]

## Part 1 

In [ ]:
sorted_data = [sorted(ls) for ls in zip(*data)]
res_a = sum(abs(x - y) for x, y in zip(*sorted_data))

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [2]:
x_counted = collections.Counter(x for x, _ in data)
y_counted = collections.Counter(y for _, y in data)

NameError: name 'collections' is not defined

In [ ]:
res_b = sum(xk * xv * y_counted[xk] for xk, xv in x_counted.items())

In [ ]:
puzzle.answer_b = res_b

# Day 2

In [17]:
puzzle = Puzzle(2024, 2)


In [12]:
data = [list(map(int, line.split(" "))) for line in puzzle.input_data.split("\n")]

In [21]:
data_test = """7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9"""

In [22]:
data_test = [list(map(int, line.split(" "))) for line in data_test.split("\n")]

In [23]:
data_test

[[7, 6, 4, 2, 1],
 [1, 2, 7, 8, 9],
 [9, 7, 6, 2, 1],
 [1, 3, 2, 4, 5],
 [8, 6, 4, 4, 1],
 [1, 3, 6, 7, 9]]

## Part 1 

In [5]:
def valid_ascend(seq):
    return all(a < b < a + 4 for a, b in itertools.pairwise(seq))

def valid_descend(seq):
    return all(a > b > a - 4 for a, b in itertools.pairwise(seq))
    
def valid_sequence(seq):
    return valid_ascend(seq) or valid_descend(seq)


In [6]:
res_a = sum(valid_sequence(sequence) for  sequence in data)
res_a

NameError: name 'data' is not defined

In [112]:
puzzle.answer_a = res_a

## Part 2

In [160]:
def valid_ascend_b(seq):
    for i, (a, b) in enumerate(itertools.pairwise(seq)):
        if not (a < b < a + 4):
            return False, i
    return True, len(seq)
            

def valid_descend_b(seq):
    for i, (a, b) in enumerate(itertools.pairwise(seq)):
        if not (a > b > a - 4):
            return False, i
    return True, len(seq)

def valid_sequence_b(sequence):
    valid, i = valid_ascend_b(sequence)
    if valid:
        return True

    for offset in range(2):
        valid, _ = valid_ascend_b(sequence[i - 1 + offset: i + offset] + sequence[i + 1 + offset:])
        if valid:
            return True

    valid, i = valid_descend_b(sequence)
    if valid:
        return True
        
    for offset in range(2):
        valid, _ = valid_descend_b(sequence[i - 1 + offset: i + offset] + sequence[i + 1 + offset:])
        if valid:
            return True
            
    return False
    

In [161]:
res_b = sum(valid_sequence_b(sequence) for sequence in data)
res_b

604

In [139]:
puzzle.answer_b = res_b

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 2! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 3

In [76]:
puzzle = Puzzle(2024, 3)

In [19]:
data_test = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

## Part 1 

In [54]:
mul_regex = re.compile("mul\((?P<X>\d{1,3}),(?P<Y>\d{1,3})\)")
def parse_mul(data):
    return sum(int(m["X"])*int(m["Y"]) for m in re.finditer(mul_regex, data))

In [57]:
res_a = parse_mul(puzzle.input_data)
res_a

175700056

In [52]:
puzzle.answer_a = res_a

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## Part 2

In [239]:
remove_dont_regex = re.compile(r"don't\(\).*?(?=(do(n't)?\(\)|$))", re.S)

In [249]:
re.match(re.compile(".*"), "\n")

<re.Match object; span=(0, 1), match='t'>

In [250]:
data_test = "don't()don't()bdon't()cdo()adon't()\ndo()ddodon't()d"

In [251]:
for m in re.finditer(remove_dont_regex, data_test):
    print(m.group())
    print()

don't()

don't()b

don't()c

don't()


don't()d



In [257]:
data_do = re.sub(remove_dont_regex, "", puzzle.input_data)

In [258]:
answer_b = parse_mul(data_do)
answer_b

71668682

In [248]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 3! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 4

In [5]:
puzzle = Puzzle(2024, 4)

In [85]:
data_test = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
grid_test = np.array([list(line) for line in data_test.split("\n")])
grid = np.array([list(line) for line in puzzle.input_data.split("\n")])

## Part 1 

In [102]:
def count_occurence(lst: list[str], word: str) -> int:
    lst_str = "".join(lst)
    # print(f"{lst_str:10} -> {word}: {lst_str.count(word)} {word[::-1]}: {lst_str.count(word[::-1])}")
    return lst_str.count(word) + lst_str.count(word[::-1])

In [112]:
def get_all_word_occurence(grid, word):
    occurences = 0
    x, y = grid.shape
    gird_sym = np.fliplr(grid)
    
    for xi in range(x):
        occurences += count_occurence(grid[xi, :], word)
    
    for yi in range(y):
        occurences += count_occurence(grid[:, yi], word)
    
    for di in range(-x + 1, y):
        occurences += count_occurence(grid.diagonal(di), word)
        occurences += count_occurence(gird_sym.diagonal(di), word)

    return occurences
        

In [113]:
get_all_word_occurence(grid_test, word)

18

In [114]:
answer_a = get_all_word_occurence(grid, word)
answer_a

2414

In [115]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## Part 2

In [125]:
def get_x_mas_occurences(grid):
    occurences = 0
    x, y = grid.shape
    for xi in range(1, x - 1):
        for yi in range(1, y - 1):
            if grid[xi, yi] != "A":
                continue
            corner = f"{grid[xi-1, yi-1]}{grid[xi-1, yi+1]}{grid[xi+1, yi+1]}{grid[xi+1, yi-1]}"
            occurences += corner in ["SSMM", "SMMS", "MMSS", "MSSM"]

    return occurences

In [126]:
get_x_mas_occurences(grid_test)

9

In [127]:
answer_b = get_x_mas_occurences(grid)
answer_b

1871

In [128]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 4! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 5

In [4]:
puzzle = Puzzle(2022, 5)

In [33]:
data = puzzle.input_data

In [102]:
instruction_pattern = re.compile(r"move (?P<number>\d+) from (?P<start>\d+) to (?P<end>\d+)")
class SupplyStacks:
    def __init__(self, data):
        queues, instructions = data.split("\n\n")
        # transpose
        queues = [''.join(s).strip() for s in zip(*queues.split("\n")) if re.match(r"\w+", ''.join(s).strip())]
        
        self.queues = {}
        # Create LIFO
        for q in queues:
            new_q = queue.LifoQueue()
            for s in q[-2::-1]:
                new_q.put(s)
            self.queues[q[-1]] = new_q
        
        # Parse instructions
        self.instructions = [instruction_pattern.match(instruction).groupdict() for instruction in instructions.split("\n")]
        
    def part_1(self):
        for instruction in self.instructions:
            for _ in range(int(instruction["number"])):
                self.queues[instruction["end"]].put(self.queues[instruction["start"]].get())
    
    def answer_a(self):
        self.part_1()
        return self._join_queues()
    
    def part_2(self):
        temporary_queue = queue.LifoQueue()
        for instruction in self.instructions:
            for _ in range(int(instruction["number"])):
                temporary_queue.put(self.queues[instruction["start"]].get())
                
            for _ in range(int(instruction["number"])):
                self.queues[instruction["end"]].put(temporary_queue.get())
                
    def answer_b(self):
        self.part_2()
        return self._join_queues()
    
    def _join_queues(self):
        return "".join(q.get() for q in self.queues.values())

## Part 1 

In [96]:
ss = SupplyStacks(data)

In [94]:
answer_a = ss.answer_a()
answer_a

'FJSRQCFTN'

In [89]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [103]:
ss = SupplyStacks(data)

In [104]:
answer_b = ss.answer_b()
answer_b

'CJVLJQPHS'

In [105]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 5! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 6

In [12]:
puzzle = Puzzle(2022, 6)

In [13]:
data = puzzle.input_data

## Part 1 

In [15]:
def get_position_message(data, length):
    for idx, chars in enumerate(more_itertools.sliding_window(data, length)):
        if len(set(chars)) == length:
            return idx + length
    
    raise ValueError("should not happen")

In [16]:
answer_a = get_position_message(data, 4)
answer_a

1912

In [16]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [17]:
answer_b = get_position_message(data, 14)
answer_b

2122

In [18]:
puzzle.answer_b = answer_b

# Day 7

In [3]:
puzzle = Puzzle(2022, 7)

In [4]:
data = puzzle.input_data.split("\n")

In [6]:
directrories_score = collections.defaultdict(int)
actual_dir = collections.deque("/")

for line in data:
    match line.split():
        # commands
        case '$', 'ls':
            pass
        case '$', 'cd', '/':
            actual_dir = collections.deque("/")
        case '$', 'cd', '..':
            actual_dir.pop()
        case '$', 'cd', dir_name:
            actual_dir.append(f"{dir_name}/")
        # after ls
        case 'dir', _:
            pass
        case size, _:
            # Accumulate for concatening all elements of actual_dir
            for directory in itertools.accumulate(actual_dir):
                directrories_score[directory] += int(size)

## Part 1 

In [8]:
answer_a = sum(val for val in directrories_score.values() if val <= 100000)
answer_a

1297683

In [109]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [9]:
answer_b = min(val for val in directrories_score.values() if 30000000 <= 70000000 - directrories_score['/'] + val)
answer_b

5756764

In [10]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 7! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 8

In [195]:
puzzle = Puzzle(2022, 8)

In [122]:
data = np.array([[int(number) for number in line] for line in puzzle.input_data.split("\n")])

In [123]:
len_x, len_y = data.shape

## Part 1 

In [136]:
total_tree = np.ones((len_x,len_y), dtype=int)
total_tree[1:-1,1:-1] = 0

for x in range(1, len_x - 1):
    for y in range(1, len_y - 1):
        number = data[x, y]
        if (
            np.all(data[:x, y] < number)
            or np.all(data[x+1:, y] < number)
            or np.all(data[x, :y] < number)
            or np.all(data[x, y+1:] < number)
         ):
            total_tree[x, y] = 1

In [137]:
answer_a = np.sum(total_tree)
answer_a

1829

In [121]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [193]:
total_tree = np.ones((len_x,len_y), dtype=int)
total_tree[1:-1,1:-1] = 0

for x in range(1, len_x-1):
    for y in range(1, len_y-1):
        number = data[x, y]
        
        a = 0
        for new_n in data[x-1::-1, y]:
            a += 1
            if new_n >= number:
                break
                
        b = 0
        for new_n in data[x+1:, y]:
            b += 1
            if new_n >= number:
                break
        
        c = 0
        for new_n in data[x, y-1::-1]:
            c += 1
            if new_n >= number:
                break
                
        d = 0
        for new_n in data[x, y+1:]:
            d += 1
            if new_n >= number:
                break
                
        total_tree[x, y] = a * b * c * d       

In [194]:
answer_b = np.max(total_tree)
answer_b

291840

In [192]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 8! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 9

In [2]:
puzzle = Puzzle(2022, 9)

In [288]:
data = puzzle.input_data

In [289]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def as_tuple(self):
        return self.x, self.y
    
    @classmethod
    def from_tuple(cls, coord):
        return Point(*coord)
                   
    @classmethod
    def create_vector(cls, direction):
        match direction:
            case "U":
                return cls(1, 0)
            case "R":
                return cls(0, 1)
            case "D":
                return cls(-1, 0)
            case "L":
                return cls(0, -1)
            case "_":
                raise ValueError("unkown direction")
        
    def one_tail_near(self, other_point):
        return abs(self.x - other_point.x) <= 1 and abs(self.y - other_point.y) <= 1
    
    def sign(self):
        return Point(np.sign(self.x), np.sign(self.y))
    
    def __sub__(self, other_point):
        return Point(self.x - other_point.x, self.y - other_point.y)
    
    def __isub__(self, other_point):
        return self - other_point
    
    def __neg__(self):
        return Point(-self.x, -self.y)
    
    def __add__(self, other_point):
        return Point(self.x + other_point.x, self.y + other_point.y)
    
    def __iadd__(self, other_point):
        return self + other_point
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Point{self}"

In [290]:
class RopeGame:
    def __init__(self, data, tail_lenght=1, start=(0,0)):
        self.instructions = [line.split(" ") for line in data.split("\n")]
        self.knots = [Point.from_tuple(start) for _ in range(tail_lenght + 1)]
        self.knots_positions = [set([start]) for _ in range(tail_lenght + 1)]
    
    def parse_instruction(self, instruction):
        direction, value = instruction
        vector = Point.create_vector(direction)
        
        for _ in range(int(value)):
            self.update_knot(0, self.knots[0] + vector)
            
            for idx, knot in list(enumerate(self.knots))[1:]:
                prev_knot = self.knots[idx-1]
                if knot.one_tail_near(prev_knot):
                    break
                
                deplacement = (prev_knot - knot).sign()
                self.update_knot(idx, knot + deplacement)
                
    def run(self):
        for instruction in self.instructions:
            self.parse_instruction(instruction)
            
    def update_knot(self, pos, new_knot):
        self.knots[pos] = new_knot
        self.knots_positions[pos].add(new_knot.as_tuple)
    
    @property
    def answer_a(self):
        return len(self.knots_positions[1])
    
    @property
    def answer_b(self):
        return len(self.knots_positions[-1])
    

## Part 1 

In [291]:
rg = RopeGame(data)
rg.run()
rg.answer_a

6057

In [292]:
puzzle.answer_a = rg.answer_a

## Part 2

In [293]:
rg = RopeGame(data, 9)
rg.run()

In [294]:
rg.answer_a

6057

In [295]:
rg.answer_b

2514

In [296]:
puzzle.answer_b = rg.answer_b

# Day 10

In [155]:
puzzle = Puzzle(2022, 10)

In [156]:
data = puzzle.input_data.split("\n")

In [157]:
def test_cycle(x, cycle):
    if cycle in {20, 60, 100, 140, 180, 220}:
        print(cycle, x)
        return cycle * x
    return 0

## Part 1 

In [158]:
x = 1
cycle = 0
answer_a = 0

for instruction in data:
    match instruction.split(" "):
        case ["noop"]:
            cycle +=1
            answer_a += test_cycle(x, cycle)
        case "addx", value:
            cycle += 1
            answer_a += test_cycle(x, cycle)
            cycle += 1
            answer_a += test_cycle(x, cycle)
            x += int(value)
        case _:
            print(instruction)
            raise ValueError("")
answer_a

20 29
60 17
100 21
140 21
180 21
220 17


14160

In [107]:
puzzle.answer_a = answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [159]:
def draw(x, cycle):
    if x <= (cycle % 40) <= x+2:
        return "#"
    return "."

In [160]:
def print_map(answer_b):
    for chunk in more_itertools.batched(answer_b, 40):
        print("".join(chunk))

In [161]:
x = 1
cycle = 0
answer_b = []

for instruction in data:
    match instruction.split(" "):
        case ["noop"]:
            cycle +=1
            answer_b.append(draw(x, cycle))
        case "addx", value:
            cycle += 1
            answer_b.append(draw(x, cycle))
            cycle += 1
            answer_b.append(draw(x, cycle))
            x += int(value)
        case _:
            print(instruction)
            raise ValueError("")


In [162]:
print_map(answer_b)

###....##.####.###..###..####.####..##.#
#..#....#.#....#..#.#..#.#....#....#..#.
#..#....#.###..#..#.#..#.###..###..#....
###.....#.#....###..###..#....#....#...#
#.#..#..#.#....#.#..#....#....#....#..#.
#..#..##..####.#..#.#....####.#.....##..


In [154]:
puzzle.answer_b = "RJERPEFC"

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 10! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 11

In [76]:
puzzle = Puzzle(2022, 11)

In [77]:
data = puzzle.input_data

In [67]:
data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [101]:
class Monkey:
    def __init__(self, name,items, operation, test_val, monkey_true, monkey_false):
        self.item_inspected = 0
        self.name = name,
        self.items = items
        self.operation = operation
        self.test_val = test_val
        self.monkey_true = monkey_true
        self.monkey_false = monkey_false
    
    def inspect(self, old, p1=True):
        self.item_inspected += 1
        item = eval(self.operation)
        if p1:
            item = item // 3
        return item
        
    def test(self, item):
        monkey_dest = self.monkey_true if item % self.test_val == 0 else self.monkey_false
        return monkey_dest, item
    
    def add_item(self, item):
        self.items.append(item)
    
    def do_round(self, p1=True):
        for item in self.items:
            item = self.inspect(item, p1=p1)
            yield self.test(item)
        self.items = []

In [131]:
class MonkeyManager:
    def __init__(self, data):
        # Create monkey
        self.monkeys = []
        self.test_total = 1
        self.data = data
        
        self.create()
    
    def create(self):

        for monkey_data in self.data.split("\n\n"):
            name,  items, operation, test, m_true, m_false = monkey_data.split("\n")
            name,  items, operation, test, m_true, m_false = monkey_data.split("\n")
            name = name.strip()
            items = list(map(int, number_pattern.findall(items)))
            operation = operation.split("= ")[1]
            test = int(number_pattern.search(test).group())
            self.test_total *= test
            m_true = int(number_pattern.search(m_true).group())
            m_false = int(number_pattern.search(m_false).group())
            new_monkey = Monkey(name, items, operation, test, m_true, m_false)
            self.monkeys.append(new_monkey)
            
    def do_round(self, p1=True):
        
        for monkey in self.monkeys:
            for monkey_dest, item in monkey.do_round(p1):
                self.monkeys[monkey_dest].add_item(item % self.test_total)
                
    def answer_a(self):
        self.reset()
        for i in range(20):
            self.do_round(p1=True)
        
        return self.monkey_inspections(2)
    
    
    def answer_b(self):
        self.reset()
        for i in tqdm.tqdm(range(10000)):
            self.do_round(p1=False)
        
        return self.monkey_inspections(2)
    
    
    def monkey_inspections(self, n):
        m_i = [monkey.item_inspected for monkey in self.monkeys]
        m_i = heapq.nlargest(2, m_i)
        return functools.reduce(operator.mul, m_i, 1)
        
    def reset(self):
        self.monkeys = []
        self.test_total = 1       
        self.create()

In [132]:
mm = MonkeyManager(data)

## Part 1 

In [133]:
answer_a = mm.answer_a()
answer_a

66802

In [113]:
puzzle.answer_a = answer_a

## Part 2

In [134]:
answer_b = mm.answer_b()
answer_b

100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2329.57it/s]


21800916620

In [135]:
puzzle.answer_b = answer_b

# Day 12

In [2]:
puzzle = Puzzle(2022, 12)

In [46]:
data = puzzle.input_data

In [3]:
data = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi"""

In [47]:
def adjs(position, nodes):
    x, y = position
    adjs = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
    return [adj for adj in adjs if adj in nodes]

In [48]:
def distance(pos1, pos2, nodes_weight):
    return nodes_weight[pos2] - nodes_weight[pos1]

In [49]:
base_nodes = np.array([[letter for letter in line] for line in data.split("\n")])
nodes_weight = np.array([[ord(letter) - ord("a") for letter in line] for line in data.split("\n")])

In [50]:
start = np.where(nodes_weight == ord("S") - ord("a"))
start = next(zip(start[0], start[1]))
nodes_weight[start] = 0

In [51]:
end = np.where(nodes_weight == ord("E") - ord("a"))
end = next(zip(end[0], end[1]))
nodes_weight[end] = ord("z") - ord("a")

In [52]:
nodes = list(np.ndindex(*nodes_weight.shape))

In [53]:
distances = {
    node: {adj: 1 for adj in adjs(node, nodes) if distance(node, adj, nodes_weight) <= 1} for node in nodes
}

In [55]:
G = nx.DiGraph()

In [27]:
G.add_nodes_from(nodes)

In [56]:
for node, adjs in distances.items():
    for adj, dist in adjs.items():
        G.add_edge(node, adj, weight=dist)

## Part 1 

In [57]:
answer_a = nx.shortest_path_length(G, start, end)
answer_a

425

In [58]:
puzzle.answer_a = answer_a

## Part 2

In [59]:
starts = np.where(nodes_weight == 0)
starts

(array([ 0,  0,  0, ..., 40, 40, 40]),
 array([  0,  18,  19, ..., 111, 112, 113]))

In [60]:
dists = []
for start in zip(starts[0], starts[1]):
    try:
        dist = nx.shortest_path_length(G, start, end)
        dists.append(dist)
    except nx.NetworkXNoPath:
        continue
        
answer_b = min(dists)
answer_b


418

In [42]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 12! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 13

In [9]:
puzzle = Puzzle(2022, 13)

In [10]:
signals_pairs = [tuple(map(eval, line.split("\n"))) for line in puzzle.input_data.split("\n\n")]

## Part 1 

In [28]:
def parse_pair(p1, p2):
    """ -1 --> a > b
    0 --> a == b
    1 --> a < b
    
    """
    if isinstance(p1, int) and isinstance(p2, int):
        return np.sign(p2 - p1)

    for e1, e2 in zip(p1, p2):
        match e1, e2:
            case int(), list():
                e1 = [e1]
            case list(), int():
                e2 = [e2]
            
        if (res := parse_pair(e1, e2)) != 0:
            return res
    
    return np.sign(len(p2) - len(p1))

In [29]:
res = sum(idx + 1 for idx, pair in enumerate(signals_pairs) if parse_pair(*pair) != -1)

In [30]:
res

5852

In [62]:
puzzle.answer_a = res

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [14]:
signals = [eval(line) for line in puzzle.input_data.split("\n") if line]

In [15]:
s1 = [[2]]
signals.append(s1)
s2 = [[6]]
signals.append(s2)

In [16]:
def compare(l1, l2):
    res = parse_pair(l1, l2)
    return -1 if res != -1 else 1

In [17]:
signals = sorted(signals, key=functools.cmp_to_key(compare))

In [23]:
filter_signals = [idx + 1 for idx, signal in enumerate(signals) if signal == s1 or signal == s2]
tot = functools.reduce(operator.mul, filter_signals, 1)

In [24]:
tot

24190

In [132]:
puzzle.answer_b = tot

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 13! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 14

In [100]:
puzzle = Puzzle(2022, 14)

In [158]:
data = puzzle.input_data

In [180]:
data_test = """498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9"""

In [ ]:
sand_entry_x, send_entry_y = sand_entry = (500, 0)

In [188]:
def get_cave_extremity(cave, p1=True):
    y_min = send_entry_y
    y_max = max(map(lambda t: t[1], cave)) + (0 if p1 else 2)
    x_min = min(map(lambda t: t[0], cave))
    x_max = max(map(lambda t: t[0], cave))
    
    return (x_min, x_max), (y_min, y_max)

In [241]:
def init_cave(data, p1=True):
    coords = [[tuple(map(int, coord.split(","))) for coord in line.split(" -> ")] for line in data.split("\n")]
    cave = collections.defaultdict(str)

    def get_numbers_inside(n1, n2):
        return range(min(n1, n2), max(n1, n2) + 1)

    for coord in coords:

        for (x1, y1), (x2, y2) in itertools.pairwise(coord):
            if x1 == x2:
                for yi in get_numbers_inside(y1, y2):
                    cave[(x1, yi)] = "#"
            # y1 == y2
            else:
                for xi in get_numbers_inside(x1, x2):
                    cave[(xi, y1)] = "#"
                    
    if not p1:
        (x_min, x_max), (y_min, y_max) = get_cave_extremity(cave, p1=p1)
        boottom_half_lenght = y_max - y_min
        
        for xi in range(- boottom_half_lenght, boottom_half_lenght + 1):
            cave[(xi + sand_entry_x, y_max)] = "#"
        
    return cave



In [242]:
np.set_printoptions(linewidth=150)
def print_cave(cave, p1=True):
    cave = dict(cave)
    (x_min, x_max), (y_min, y_max) = get_cave_extremity(cave, p1=p1)
    representation = np.chararray((x_max - x_min + 1, y_max - y_min + 1), unicode=True)
    representation[:, :] = "."
    for (x, y), value in cave.items():
        representation[x - x_min, y] = value
    representation[sand_entry_x - x_min, y_min] = "x"
    print(representation.T)

In [243]:
cave_test = init_cave(data_test, p1=False)
print_cave(cave_test)

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' 'x' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '#' '#' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '#' '#' '.' '.' '.' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '#' '#' '#' '#' '#' '#' '#' '#' '#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' 

## Part 1

In [257]:
def compute(data, p1=True):
    cave = init_cave(data, p1=p1)
    time = -1
    (x_min, x_max), (y_min, y_max) = get_cave_extremity(cave, p1=p1)
    while True:
        time += 1
        sand_pos_x, sand_pos_y = sand_entry
        
        if cave[sand_entry] == "o":
            return cave, time - 1
        
        while True:
            under = (sand_pos_x, sand_pos_y + 1)

            match cave[under]:
                case "#" | "o":
                    under_left  = (sand_pos_x - 1, sand_pos_y + 1)
                    under_right = (sand_pos_x + 1, sand_pos_y + 1)

                    if cave[under_left] == "":
                        sand_pos_x, sand_pos_y = under_left
                    elif cave[under_right] == "":
                        sand_pos_x, sand_pos_y = under_right
                    else:
                        cave[(sand_pos_x, sand_pos_y)] = "o"
                        break
                case "":
                    sand_pos_x, sand_pos_y = under

            if sand_pos_y == y_max:
                return cave, time

In [258]:
cave, time = compute(data_test)
print(f"{time=}")
print_cave(cave)

time=24
[['.' '.' '.' '.' '.' '.' '.' 'x' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '' 'o' '.' '.' '.']
 ['.' '.' '.' '.' '.' '' 'o' 'o' 'o' '.' '.']
 ['.' '.' '.' '.' '' '#' 'o' 'o' 'o' '#' '#']
 ['.' '.' '.' '' 'o' '#' 'o' 'o' 'o' '#' '.']
 ['.' '.' '' '#' '#' '#' 'o' 'o' 'o' '#' '.']
 ['.' '.' '' '.' '.' 'o' 'o' 'o' 'o' '#' '.']
 ['.' '' 'o' '.' 'o' 'o' 'o' 'o' 'o' '#' '.']
 ['' '#' '#' '#' '#' '#' '#' '#' '#' '#' '.']]


In [246]:
_, answer_a = compute(data)
answer_a

774

In [252]:
puzzle.answer_a = answer_a

## Part 2

In [69]:
cave, time = compute(data_test, p1=False)
print(f"{time=}")
print_cave(cave, p1=False)

NameError: name 'data_test' is not defined

In [250]:
_, answer_b = compute(data, p1=False)
answer_b

22499

In [253]:
puzzle.answer_b = answer_b

# Day 15

In [2]:
puzzle = Puzzle(2022, 15)

In [3]:
data = puzzle.input_data

In [4]:
data_test = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

In [5]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def as_tuple(self):
        return self.x, self.y
    
    @classmethod
    def from_tuple(cls, coord):
        return Point(*coord)
      
    def manathan_distance(self, other_point):
        return abs(self.x - other_point.x) + abs(self.y - other_point.y)
    
    def det(self, point):
        return self.x * point.y - self.y * point.x
    
    def __sub__(self, other_point):
        return Point(self.x - other_point.x, self.y - other_point.y)
    
    def __isub__(self, other_point):
        return self - other_point
    
    def __neg__(self):
        return Point(-self.x, -self.y)
    
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __iadd__(self, other_point):
        return self + other_point
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Point{self}"
    
class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        
    def det(self):
        return self.p1.det(self.p2)
    
    def intersection(self, line):
        xdiff = Point(self.p1.x - self.p2.x, line.p1.x - line.p2.x)
        ydiff = Point(self.p1.y - self.p2.y, line.p1.y - line.p2.y)
        
        div = xdiff.det(ydiff)
        if div == 0:
            return False
        
        d = Point(self.det(), line.det())
        x = d.det(xdiff) // div
        y = d.det(ydiff) // div
        
        return Point(x, y)

class Sensor:
    def __init__(self, sensor, beacon):
        self.sensor = sensor
        self.beacon = beacon
        self.distance = sensor.manathan_distance(beacon)
        self.outer_distance = self.distance + 1
        self.lines = self.possible_outer_lines()

        
    def possible_outer_points(self):
        x, y = self.sensor.as_tuple
        return [
            Point(x - self.outer_distance, y),
            Point(x, y - self.outer_distance),
            Point(x + self.outer_distance, y),
            Point(x, y + self.outer_distance)
        ]
    
    def possible_outer_lines(self):
        points_i = self.possible_outer_points()
        points_j = points_i[1:] + [points_i[0]]
        
        return [Line(pi, pj) for (pi, pj) in zip(points_i, points_j)]
            
    def point_possible(self, point):
        return self.sensor.manathan_distance(point) > self.distance
    
    def outer_intersections_sensor(self, other_sensor, start, end):
        for line_i in self.lines:
            for line_j in other_sensor.lines:
                if (intersection := line_i.intersection(line_j)):
                    if start <= intersection.x <= end and start <= intersection.y <= end:
                        yield intersection       

In [6]:
number_pattern = re.compile("-?\d+")
def parse_data(data):
    sensors_beacons = []
    for line in data.split("\n"):
        s_x, s_y, b_x, b_y = map(lambda match: int(match.group()), number_pattern.finditer(line))
        sensor, beacon = Point(s_x, s_y), Point(b_x, b_y)
        sensors_beacons.append((sensor, beacon, sensor.manathan_distance(beacon)))
        
    return sensors_beacons

In [7]:
number_pattern = re.compile("-?\d+")
def parse_data2(data):
    sensors = []
    for line in data.split("\n"):
        s_x, s_y, b_x, b_y = map(lambda match: int(match.group()), number_pattern.finditer(line))
        sensor, beacon = Point(s_x, s_y), Point(b_x, b_y)
        sensors.append(Sensor(sensor, beacon))
    return sensors

## Part 1

In [120]:
def span(sensor, beacon, m_s_b, y):
    """
    find x
    sensor.manathan(x, y) = sensor.manathan(beacon)
    abs(sx - x) + abs(sy - y) = sensor.manathan(beacon)
    abs(sx - x) = sensor.manathan(beacon) - abs(sy - y) = z avec z < 0
    sx - x = z AND sx - x = -z
    x = sx -z and x = sx + z
    """
    z = m_s_b - abs(sensor.y - y)
    if z < 0:
        return range(0)
    
    x1, x2 = sensor.x - z, sensor.x + z 
    
    return range(min(x1, x2), max(x1, x2) + 1)

def span_2(sensor, beacon, m_s_b, y):
    """
    find x
    sensor.manathan(x, y) = sensor.manathan(beacon)
    abs(sx - x) + abs(sy - y) = sensor.manathan(beacon)
    abs(sx - x) = sensor.manathan(beacon) - abs(sy - y) = z avec z < 0
    sx - x = z AND sx - x = -z
    x = sx -z and x = sx + z
    """
    z = m_s_b - abs(sensor.y - y)
    if z < 0:
        return False
    
    x1, x2 = sensor.x - z, sensor.x + z 
    
    return [min(x1, x2), max(x1, x2)]

In [151]:
def part_1_2(data, y):
    line_y = []
    b_s = set()
    for sensor, beacon, m_s_b in parse_data(data):
        if sensor.y == y:
            b_s.add(sensor.x)
        if beacon.y == y:
            b_s.add(beacon.x)
        if (r := span_2(sensor, beacon, m_s_b, y)):
            line_y.append(r)
            
    
    line_y.sort(key=lambda interval: interval[0])
    merged = [line_y[0]]
    for current in line_y:
        previous = merged[-1]
        if current[0] <= previous[1]:
            previous[1] = max(previous[1], current[1])
        else:
            merged.append(current)
            
    return sum(r_e - r_s + 1 for r_s, r_e in merged) - sum(r_s <= s <= r_e for r_s, r_e in merged for s in b_s)


In [142]:
def part_1_1(data, y):
    line_y = set()
    b_s = set()
    for sensor, beacon, m_s_b in parse_data(data):
        if sensor.y == y:
            b_s.add(sensor.x)
        if beacon.y == y:
            b_s.add(beacon.x)
        line_y |= set(span(sensor, beacon, m_s_b, y))
    line_y -= b_s
    return len(line_y)

In [143]:
part_1_2(data_test, 10)

26


26

In [154]:
part_1_1(data_test, 10)

26

In [145]:
part_1_1(data, 2000000) == part_1_2(data, 2000000)

4724228


True

In [146]:
%%timeit
part_1_1(data, 2000000)

1.28 s ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [152]:
%%timeit
part_1_2(data, 2000000)

135 µs ± 7.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [156]:
answer_a = part_1_2(data, 2000000)
puzzle.answer_a = answer_a

## Part 2

In [14]:
def find_outer_points(sensor, m_s_b):
    outer_distance = m_s_b + 1
    sx, sy = sensor.as_tuple
    outers = []
    for i in range(outer_distance):
        yield Point(sx - outer_distance + i, sy + i)
        yield Point(sx - outer_distance + i, sy - i)
        yield Point(sx + i, sy + outer_distance - i)
        yield Point(sx - i, sy + outer_distance - i)

In [15]:
def all_possible_point(sensors_beacons, start, end):
    for sensor, _, m_s_b in sensors_beacons:
        for point in find_outer_points(sensor, m_s_b):
            if start <= point.x <= end and start <= point.y <= end:
                yield point
                
def compute(data, start=0, end=20):
    sensors_beacons = parse_data(data)
    
    for point in tqdm.tqdm(all_possible_point(sensors_beacons, start, end)):
        for sensor, _, m_s_b in sensors_beacons:
            if sensor.manathan_distance(point) <= m_s_b:
                break
        else:
            print(point)
            return point

In [ ]:
compute(data_test, 0, 20)

In [17]:
res = compute(data, 0, 4000000)

41489065it [02:34, 268691.78it/s]

(3405562, 3246513)


In [19]:
answer_b = res.x*4000000 + res.y
answer_b

13622251246513

In [52]:
puzzle.answer_b = answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 15! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [ ]:
def all_possible_point_with_line(sensors, start, end):
    for si, sj in itertools.combinations(sensors, 2):
        yield from si.outer_intersections_sensor(sj, start, end)

In [22]:
def compute2(data, start=0, end=4000000):
    sensors = parse_data2(data)
    for point in all_possible_point_with_line(sensors, start, end):
        for sensor in sensors:
            if not sensor.point_possible(point):
                break
        else:
            return point

In [23]:
compute2(data_test, 0, 20)

Point(14, 11)

In [24]:
%%timeit
res2 = compute2(data, 0, 4000000)

10.5 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
res2 = compute2(data, 0, 4000000)
answer_b_2 = res2.x*4000000 + res2.y

In [26]:
answer_b_2 == answer_b

True